## Inheritance

**Inheritance provides**
- reusability
- extensibility
- avoids code duplication

In the last lecture, we saw how classes bring structure and readability to our code

Specifically, Parent class defines the interface and structure first.
Then, child classes implement it accordingly

Moving forward from our geometry example, here is the parent class

In [6]:
from abc import ABC, abstractmethod
from math import pi

# Parent Class
class Shape(ABC):
    @abstractmethod
    def draw(self):
        pass
    
    @abstractmethod
    def area(self):
        pass

Below we see how Child classes - Circle and Square implement the abstract methods as per their definitions

In [3]:
# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']

    def __repr__(self):
        return f'Circle(r={self.radius})'
        
    def draw(self):
        return "Drawing a circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']

    def __repr__(self):
        return f'Square(a={self.side})'
        
    def draw(self):
        return "Drawing a square"
        
    @property
    def area(self):
        return self.side ** 2

## Taking Inheritance Further

Suppose we want to create a class which has a combination of two geometries e.g.
- Circle with a hole
- Square with a hole

(Assume that a hole is always of circular shape)

One obvious approach is using Inheritance which can be read as
- CircleWithHole is a circle with a hole (is-A relation)
- SquareWithHole is a square with a hole (is-A relation)

In [29]:
# CircleWithHole -> is-A -> Circle
class CircleWithHole(Circle):
    def __init__(self,**kwargs):
        super().__init__(radius=kwargs['radius'])
        self.hole_radius = kwargs['hole_radius']

    def __repr__(self):
        return f"CircleWithHole(r={self.radius},r'={self.hole_radius})"
        
    def draw(self):
        return "Drawing a circle with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [30]:
# CLIENT CODE
radius = 2
side = 2
hole_radius = 1

geom = CircleWithHole(radius=radius,hole_radius=hole_radius)
print(f'{geom.draw()} of area {geom.area}')

Drawing a circle with hole of area 9.42477796076938


# Problem with Inheritance

### 1. Class Hierarchy introduces Coupling
We can see how CircleWithHole sub-class has to know about its super class in init method
If we changed the definition of super class, it has to be changed in child class as well.
This is called coupling

To understand this, let's change the definition of `Circle` class to take another parameter `color` and let's also modify the corresponding `draw` method

In [50]:
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        self.color = kwargs['color']
        
    def draw(self):
        return f"Drawing a {self.color} circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

To consume this change in super class `Circle`, we also need to change it in the sub-class `CircleWithHole`.

This coupling issue can become a big pain in production code

In [51]:
class CircleWithHole(Circle):
    def __init__(self,**kwargs):
        super().__init__(radius=kwargs['radius'],color=kwargs['color'])
        self.hole_radius = kwargs['hole_radius']
        
    def __repr__(self):
        return f"CircleWithHole(r={self.radius},r'={self.hole_radius})"
        
    def draw(self):
        return f"Drawing a {self.color} circle with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [55]:
# CLIENT CODE
radius = 2
hole_radius = 1
color = 'red'

geom = CircleWithHole(radius=radius,color=color,hole_radius=hole_radius)
geom.draw()

'Drawing a red circle with hole'

### 2. Indeterminate explosion of child classes
Suppose we want to create more hieararchy of sub-classes e.g.
- SquareWithHole
- HexagonWithHole
- CircleWithHoleAndPadding
- etc.

We would need to implement each and every child classes which will involve lot of duplicate code.

In the example below, we implement `SquareWithHole` sub-class. 

Notice how the area method is completely same, still it needs to be implemented in each child class

In [31]:
# SquareWithHole -> is-A -> Square
class SquareWithHole(Square):
    def __init__(self,**kwargs):
        super().__init__(side=kwargs['side'])
        self.hole_radius = kwargs['hole_radius']

    def __repr__(self):
        return f"SquareWithHole(a={self.side},r'={self.hole_radius})"
        
    def draw(self):
        return "Drawing a square with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [32]:
# CLIENT CODE
radius = 2
side = 2
hole_radius = 1

# Creates SquareWithHole
geom = SquareWithHole(side=side,hole_radius=hole_radius)
geom

SquareWithHole(a=2,r'=1)

In [33]:
geom.area

0.8584073464102069

# Introducing Composition

Composition is a way to compose new child classes by writing the business logic at one place.
- reduces coupling and
- avoids code duplication

Once again, let's have our parent geometries defined as below

In [39]:
from abc import ABC, abstractmethod

# Parent Class
class Shape(ABC):
    
    @abstractmethod
    def area(self):
        pass

# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def __repr__(self):
        return f'Circle(r={self.radius})'
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Square(a={self.side})'
        
    @property
    def area(self):
        return self.side ** 2

class Hexagon(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Hexagon(a={self.side})'
    
    @property
    def area(self):
        factor = 0.5 *(3 ** 1.5)
        return factor * self.side ** 2

class Rectangle(Shape):
    def __init__(self,**kwargs):
        self.length = kwargs['length']
        self.width = kwargs['width']
        
    def __repr__(self):
        return f'Rectangle(l={self.length}, b={self.width})'
    
    @property
    def area(self):
        return self.length * self.width

To create new geometries which are a hierarchy / combination / mixture of the above base geometries, we can use Composition instead of Inheritance as shown below.

Below we define a `CustomShape` parent class which takes two shapes as init arguments
- `base_shape` which can be any basic geometry e.g. Circle, Square, Hexagon etc.
- `hole_shape` which is assumed to be circular for simplicity but can be anything

In [44]:
class CustomShape:
    def __init__(self,**kwargs):
        self.base_shape: Shape = kwargs['base_shape']
        self.hole_shape: Shape = kwargs['hole_shape']

    def __repr__(self):
        return f'CustomShape(base_shape={self.base_shape}, hole_shape={self.hole_shape})'
        
    @property
    def area(self):
        return self.base_shape.area - self.hole_shape.area

# Inheritance vs Composition

## 1. Solving Exponential Explosion of Child Classes Using Composition
- There is no need to create multiple sub-classes anymore
- Using Composition, it is very easy to create any new class by just defining the base shape and hole shape.
- There is no code duplication

### Case 1. Composing SquareWithHole as a CustomShape

Composition can be attributed as has-A relation.

As shown below, `SquareWithHole` can easily be composed using `CustomShape` class as follows
- `SquareWithHole` has a Square base shape and a Circular hole shape

In [37]:
# CLIENT CODE
side = 2
hole_radius = 1

# Creates SquareWithHole
geom = CustomShape(base_shape=Square(side=side),hole_shape=Circle(radius=hole_radius))
# Finding area
geom, geom.area

(CustomShape(base_shape=Square(a=2), hole_shape=Circle(r=1)),
 0.8584073464102069)

### Case 2. Composing CircleWithHole as a CustomShape

Similarly, `CircleWithHole` can easily be composed using `CustomShape` class as follows
- `CircleWithHole` has a Circular base shape and a Circular hole shape

In [38]:
# CLIENT CODE
radius = 2
hole_radius = 1

# Creates CircleWithHole
geom = CustomShape(base_shape=Circle(radius=radius),hole_shape=Circle(radius=hole_radius))
# Finding area
geom, geom.area

(CustomShape(base_shape=Circle(r=2), hole_shape=Circle(r=1)), 9.42477796076938)

### Case 3. Composing RectangleWithSquareHole as a CustomShape

Similarly, `RectangleWithSquareHole` can easily be composed using `CustomShape` class as follows
- `RectangleWithSquareHole` has a Rectangle base shape and a Square hole shape

In [40]:
# CLIENT CODE
length = 3
width = 2
hole_side = 1

# Creates RectangleWithSquareHole
geom = CustomShape(base_shape=Rectangle(length=length,width=width),hole_shape=Square(side=hole_side))
# Finding area
geom, geom.area

(CustomShape(base_shape=Rectangle(l=3, b=2), hole_shape=Square(a=1)), 5)

### Case 4. Composing HexagonWithCircularHole as a CustomShape

Similarly, `HexagonWithCircularHole` can easily be composed using `CustomShape` class as follows
- `HexagonWithCircularHole` has a Hexagon base shape and a Circle hole shape

In [41]:
# CLIENT CODE
side = 2
hole_radius = 1

# Creates RectangleWithSquareHole
geom = CustomShape(base_shape=Hexagon(side=side),hole_shape=Circle(radius=hole_radius))
# Finding area
geom, geom.area

(CustomShape(base_shape=Hexagon(a=2), hole_shape=Circle(r=1)),
 7.250712191823471)

# Inheritance vs Composition

## 2. Solving Coupling Issue with Composition
- There is separation of concern between creation and use due to has-A relation
- Hence, there is no coupling issue

Suppose, we change our base geometry class `Circle` to include `color` and also modify the `draw` method

In [47]:
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        self.color = kwargs['color']

    def __repr__(self):
        return f'Circle(r={self.radius})'
        
    def draw(self):
        return f"Drawing a {self.color} circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

Due to this change, we saw how Inheritance yielded subsequent changes in child classes.

But here, since we have used Composition, there is no need for any code change as there is no coupling. 

As observed below, the `CustomShape` class is a separate parent class and is not inheriting from any super class.
- It is composed of two objects which are supplied in init arguments.
- The `draw` method also does not depend on super class

In [48]:
class CustomShape:
    def __init__(self,**kwargs):
        self.base_shape: Shape = kwargs['base_shape']
        self.hole_shape: Shape = kwargs['hole_shape']

    def __repr__(self):
        return f'CustomShape(base_shape={self.base_shape}, hole_shape={self.hole_shape})'

    def draw(self):
        return f'{self.base_shape.draw()} with hole'
        
    @property
    def area(self):
        return self.base_shape.area - self.hole_shape.area

In [54]:
# CLIENT CODE
radius = 2
hole_radius = 1
color = 'red'

geom = CustomShape(base_shape=Circle(radius=radius,color=color),hole_shape=Circle(radius=hole_radius,color=color))
geom.draw()

'Drawing a red circle with hole'